In [1]:
import tensorflow as tf
import os
import time
import argparse
import datetime

from SE_model_multi_attn import *
from process_data_multi import *
from evaluation_multi import *
from project_config import *

In [2]:
batch_gen = ProcessDataMulti('../data/target/audio_woZ_set01/')

model = SingleEncoderModelMultiAttn(
                                    batch_size = 2,
                                    lr = 0.001,
                                    encoder_size_audio = 10,
                                    num_layer_audio = 1,
                                    hidden_dim_audio = 10,                
                                    dr_audio = 1.0,
                                    encoder_size_text = 5,
                                    num_layer_text = 1,
                                    hidden_dim_text = 5,                
                                    dr_text = 1.0,
                                    dic_size = batch_gen.dic_size,
                                    use_glove = False
                                    )
model.build_graph()

load data : train_audio_mfcc.npy train_seqN.npy train_audio_prosody.npy train_nlp_trans.npy train_label.npy
[completed] load data
load data : dev_audio_mfcc.npy dev_seqN.npy dev_audio_prosody.npy dev_nlp_trans.npy dev_label.npy
[completed] load data
load data : test_audio_mfcc.npy test_seqN.npy test_audio_prosody.npy test_nlp_trans.npy test_label.npy
[completed] load data
[launch-multi] create audio model
[launch-audio] placeholders

[launch-audio] create gru cell

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [9]:
raw_encoder_inputs_audio, raw_encoder_seq_audio, raw_encoder_prosody, raw_encoder_inputs_text, raw_encoder_seq_text, raw_label = batch_gen.get_batch(
                        batch_size=model.batch_size,
                        data=batch_gen.train_set,
                        encoder_size_audio=model.encoder_size_audio,
                        encoder_size_text=model.encoder_size_text,
                        is_test=False
                        )

# prepare data which will be push from pc to placeholder
input_feed = {}

input_feed[model.encoder_inputs_audio] = raw_encoder_inputs_audio
input_feed[model.encoder_seq_audio] = raw_encoder_seq_audio
input_feed[model.encoder_prosody] = raw_encoder_prosody
input_feed[model.dr_prob_audio] = model.dr_audio

input_feed[model.encoder_inputs_text] = raw_encoder_inputs_text
input_feed[model.encoder_seq_text] = raw_encoder_seq_text
input_feed[model.dr_prob_text] = model.dr_text

input_feed[model.y_labels] = raw_label

print(raw_encoder_inputs_text)
print(raw_encoder_seq_text)

[array([1672,  621,    0,    0,    0]), array([1385,  787, 1879, 3558,  667])]
[2, 5]


In [10]:
r1, r2 = sess.run([model.tmp_norm,
                   model.tmp_norm],
                  input_feed)

In [11]:
r1

array([[0.42401603, 0.57598394, 0.        , 0.        , 0.        ],
       [0.16723073, 0.22598384, 0.21686387, 0.21327227, 0.17664926]],
      dtype=float32)